#### Script to download PDFs with ICES latest advice. Note the following caveats:

Some PDFs are not named exactly as the fish stock, they have a -1 or a (1) at the end of the name

There are some PDFs that are Replaced. It seems the link hasn't been updated on the dataset yet

Finally, some species in the dataset have no link to the pdf, it must be searched manually

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import webbrowser
import time

In [ ]:
# stocks we are interested in
stockW = ['cod.27.47d20',
 'bss.27.4bc7ad-h',
 'ghl.27.561214',
 'hke.27.8c9a',
 'meg.27.7b-k8abd',
 'hke.27.3a46-8abd',
 'mac.27.nea',
 'san.sa.3r',
 'sol.27.7d',
 'whg.27.47d',
 'pok.27.1-2',
 'cod.2127.1f14',
 'sol.27.20-24',
 'cod.21.1a-e',
 'her.27.3a47d',
 'nop.27.3a4',
 'cod.27.7e-k',
 'lin.27.5a',
 'her.27.irls',
 'her.27.25-2932',
 'ple.27.420',
 'had.27.7b-k',
 'ple.27.7d',
 'ple.27.7a',
 'spr.27.3a4',
 'had.27.7a',
 'spr.27.22-32',
 'had.27.5a',
 'san.sa.1r',
 'her.27.3031',
 'reb.27.1-2',
 'pok.27.3a46',
 'her.27.1-24a514a',
 'sol.27.8ab',
 'mon.27.78abd',
 'bss.27.8ab',
 'sol.27.7e',
 'lez.27.4a6a',
 'reg.27.1-2',
 'ldb.27.8c9a',
 'san.sa.4',
 'whg.27.6a',
 'cod.27.21',
 'cod.27.7a',
 'had.27.46a20',
 'lez.27.6b',
 'hom.27.2a4a5b6a7a-ce-k8',
 'ple.27.21-23',
 'cod.27.5a',
 'cap.27.1-2',
 'cod.27.22-24',
 'pok.27.5a',
 'cod.27.1-2',
 'whg.27.7b-ce-k',
 'pra.27.3a4a',
 'mon.27.8c9a',
 'reb.27.1-2',
 'sol.27.7fg',
 'whb.27.1-91214',
 'usk.27.5a14',
 'her.27.5a',
 'her.27.6a7bc',
 'had.27.6b',
 'sol.27.4',
 'san.sa.2r',
 'had.27.1-2',
 'her.27.28',
 'hom.27.9a',
 'sol.27.7a',
 'bli.27.5b67',
 'reg.27.561214',
 'meg.27.8c9a',
 'cod.27.6a',
 'her.27.20-24',
 'pra.27.1-2',
 'ple.27.7fg',
 'her.27.nirs'
 ]

In [ ]:
# load the stock assessment dataset from where we get the links to the advice
stockA = pd.read_csv("../data/stockAssesment2022/StockAssessment.csv",  names=range(138))
stockA.columns = stockA.iloc[0,:]
stockA = stockA[1:]
stockA = stockA[stockA.FishStock.isin(stockW)]
stockA = stockA[stockA.Purpose != 'Replaced']
stockA.drop_duplicates(subset=['Report'], inplace=True, keep='last')

In [ ]:
%%time
stockA['urlDown'] = ''
# go through the links and get the pdf links identified by ndownloader
for url in stockA.Report:
    if 'http' in str(url):
        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        for link in soup.find_all('a'):
            if 'ndownloader' in link.get('href'):
                stockA.loc[stockA.Report == url, 'urlDown'] = link.get('href')


In [ ]:
stockA.to_csv('../dataTemp/icesAutoAnalysis/stockPDFlinks.csv', index=False)

# read the names of the files already downloaded without extension
downloaded = [os.path.splitext(filename)[0] for filename \
              in os.listdir("../dataTemp/icesAutoAnalysis/icesPDFs")]

In [ ]:
# check what stocks missing in folder
downList = []
for stock in list(set(stockA.FishStock)):
        if stock not in downloaded:
                downList.append(stockA.urlDown[stockA.FishStock == stock].values[0])
                print(stock)

In [ ]:
for i in downList:
    webbrowser.open_new_tab(i)
    time.sleep(6)